In [ ]:
%pip install transformers
%pip install datasets
%pip install peft
%pip install bitsandbytes

  Using cached peft-0.11.1-py3-none-any.whl (251 kB)
  Using cached accelerate-0.32.1-py3-none-any.whl (314 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia

In [59]:
import torch
from torch.utils.data import  DataLoader,TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BlenderbotForCausalLM,AutoTokenizer,AdamW
from datasets import Dataset,DatasetDict
from peft import LoraConfig,TaskType,get_peft_model
# from google.colab import drive
import bitsandbytes as bnb
from sklearn.metrics import precision_score, recall_score
from nltk.translate.bleu_score import corpus_bleu
import wandb


In [60]:
wandb.init(project="QLoRA-Small-BlenderBot-Model-Casual-FineTuning")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: siddiqmoideen07. Use `wandb login --relogin` to force relogin


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [61]:
with open('T:\\OpenSourceContribution\\aXtrStuff\\neuroLang\\demoStuff\\dataStuff\\bookStuff\\osBookDoc.txt','r',encoding='utf-8', errors='ignore') as p:
    s=p.readlines()

In [62]:
s

['\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 'OPERATING SYSTEM CONCEPTS\n',
 'NINTH EDITION\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 'OPERATING SYSTEM CONCEPTS\n',
 'ABRAHAM SILBERSCHATZ\n',
 'Yale University\n',
 '\n',
 'PETER BAER GALVIN\n',
 '\n',
 'Vice President and Executive Publisher\tDon Fowley\n',
 'Executive Editor\tBeth Lang Golub\n',
 'Editorial Assistant\tKatherine Willis\n',
 'Executive Marketing Manager\tChristopher Ruel\n',
 'Senior Production Editor\tKen Santor\n',
 'Cover and title page illustrations\tSusan Cyr\n',
 'Cover Designer\tMadelyn Lesure\n',
 'Text Designer\tJudy Allan\n',
 '\n',
 '\n',
 '\n',
 '\n',
 'This book was set in Palatino by the author using LaTeX and printed and bound by Courier- Kendallville. The cover was printed by Courier.\n',
 '\n',
 'Copyright  2013, 2012, 2008 John Wiley & Sons, Inc. All rights reserved.\n',
 '\n',
 '\n',
 'No part of this publication may be reproduced, stored in a retrieval system or transmi

In [63]:
s=s[706:]

In [64]:
len(s)

20409

In [65]:
for i in range(len(s)):
    try:
        s.remove('\n')
    except:
        continue

In [86]:

for i in s:
    if(len(i)<190):
        s.remove(i)
    else :
        continue


In [87]:
len(s)

3861

In [88]:
s

['   Internally, operating systems vary greatly in their makeup, since they are organized along many different lines. The design of a new operating system is a major task. It is important that the goals of the system be well defined before the design begins. These goals form the basis for choices among various algorithms and strategies.\n',
 '   Because an operating system is large and complex, it must be created piece by piece. Each of these pieces should be a well-delineated portion of the system, with carefully defined inputs, outputs, and functions.\n',
 'An operating system is a program that manages a computers hardware. It also provides a basis for application programs and acts as an intermediary between the computer user and the computer hardware. An amazing aspect of operating systems is how they vary in accomplishing these tasks. Mainframe operating systems are designed primarily to optimize utilization of hardware. Personal computer (PC) operating systems support complex game

In [89]:
dataDict={'context':s}

In [90]:
data=pd.DataFrame(dataDict)

In [91]:
data

,context
0,"Internally, operating systems vary greatly ..."
1,Because an operating system is large and co...
2,An operating system is a program that manages ...
3,Before we can explore the details of compu...
4,Because an operating system is large and co...
...,...
3856,[Rashid and Robertson (1981)] R. Rashid and G....
3857,"[Tevanian et al. (1987b)] A. Tevanian, Jr., R...."
3858,"Figure 1.11: From Hennesy and Patterson, Comp..."
3859,Figure 12.9: From Leffler/McKusick/Karels/Qua...


In [92]:
train,val=train_test_split(data,random_state=42)

In [93]:
device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [94]:
def replace_linear_with_quantized(model):
    for name, module in model.named_children():
        if isinstance(module, torch.nn.Linear):
            quantized_layer = bnb.nn.Linear8bitLt(
                module.in_features, module.out_features, bias=module.bias is not None
            )
            setattr(model, name, quantized_layer)
        else:
            replace_linear_with_quantized(module)


In [95]:
model=BlenderbotForCausalLM.from_pretrained('facebook/blenderbot-400M-distill').to(device)
tokenizer=AutoTokenizer.from_pretrained('facebook/blenderbot-400M-distill')

In [96]:

replace_linear_with_quantized(model)


In [97]:
def prep(examples):
    inputs=[]
    mask=[]
    labels=[]
    for prompt in examples['context']:

        input=tokenizer(str(prompt),max_length=256,truncation=True,padding='max_length',return_tensors='pt')
        inputs.append(input['input_ids'])
        mask.append(input['attention_mask'])
        labels.append(input['input_ids'])
    return inputs,mask,labels

In [98]:
trainInputs, trainMasks, trainLabels =prep(train)
valinp,valmask,vallabel=prep(val)

In [99]:
trainInputs = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainInputs], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
trainMasks = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainMasks], batch_first=True, padding_value=0).to(device)
trainLabels = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainLabels], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
valinp      =   torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in valinp], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
valmask     =   torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in valmask], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
vallabel    =   torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in vallabel], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)

C:\Users\Sid\AppData\Local\Temp\ipykernel_8796\1426310643.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trainInputs = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainInputs], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
C:\Users\Sid\AppData\Local\Temp\ipykernel_8796\1426310643.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trainMasks = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainMasks], batch_first=True, padding_value=0).to(device)
C:\Users\Sid\AppData\Local\Temp\ipykernel_8796\1426310643.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.c

In [100]:
trainData=TensorDataset(trainInputs,trainMasks,trainLabels)
trainData=DataLoader(trainData,shuffle=True,batch_size=4)

In [101]:
valDataset=TensorDataset(valinp,valmask,vallabel)
valDataloader=DataLoader(valDataset,batch_size=4,shuffle=True)

In [102]:
model.config

BlenderbotConfig {
  "_name_or_path": "facebook/blenderbot-400M-distill",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "BlenderbotForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1280,
  "decoder_attention_heads": 32,
  "decoder_ffn_dim": 5120,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": true,
  "dropout": 0.1,
  "encoder_attention_heads": 32,
  "encoder_ffn_dim": 5120,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 2,
  "encoder_no_repeat_ngram_size": 3,
  "eos_token_id": 2,
  "extra_layer_norm": false,
  "extra_pos_embeddings": 0,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
 

In [103]:
lora_config = LoraConfig(
    r=64,                                # Adapter size or other fine-tuning parameter
    lora_alpha=32,                       # Adapter initialization strength
    target_modules=[
    "model.shared",                       # Shared modules across encoder and decoder
    "model.encoder.layers",               # Encoder layers for processing input sequences
    "model.encoder.self_attn.q_proj",     # Attention projection for encoder self-attention query
    "model.encoder.self_attn.v_proj",     # Attention projection for encoder self-attention value
    "model.decoder.layers.0.self_attn.q_proj", # Example of accessing a specific Linear layer
    "model.decoder.layers.0.self_attn.v_proj", # Example of accessing another specific Linear layer
],
    lora_dropout=0.05,                   # Dropout rate for LORA
    bias="none",                         # Bias setting (e.g., none, fixed, adapt)
    task_type=TaskType.CAUSAL_LM         # Task type for causal language modeling (adjust if necessary)
)


In [104]:
model=get_peft_model(model,lora_config)

In [105]:
model.print_trainable_parameters()

trainable params: 327,680 || all params: 335,859,200 || trainable%: 0.0976


In [106]:
batchSize=4
learningRate=2e-5
epochs=400

In [107]:
optimizer  = AdamW(model.parameters(),lr=learningRate)
criterion  = torch.nn.CrossEntropyLoss()

c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [108]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [109]:
import tqdm

In [110]:
wandb.watch(model)
wandb.config["learning_rate"] = learningRate
wandb.config["batch_size"] = batchSize

In [154]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
def train(model,optimizer,criterion,DataLoader):
    from tqdm.auto import tqdm
    model.train()
    tloss=0
    tcorrect=0 # Initialize a variable to store the number of correct predictions
    total_samples = 0
    all_preds = []
    all_labels = [] # Initialize a variable to store the total number of samples
    for batch,(inputs,mask,lab)in enumerate(DataLoader):
        inputs=inputs.to(device)
        mask=mask.to(device)
        label=lab.to(device)
        optimizer.zero_grad()
        outputs=model(input_ids=inputs,attention_mask=mask,labels=label)
        loss=outputs.loss
        loss.backward()
        optimizer.step()
        tloss+=loss.item()
        preds = torch.argmax(outputs.logits, dim=-1)
        
            
        all_preds.extend(preds.cpu().numpy().flatten())
        all_labels.extend(label.cpu().numpy().flatten())
        
        torch.cuda.empty_cache()
    accuracy =accuracy_score(all_labels, all_preds)


    return tloss/len(DataLoader),accuracy 

In [155]:
import torch
from sklearn.metrics import precision_score, recall_score, accuracy_score
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

def validate(model, dataloader, criterion, device, tokenizer):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    all_references = []
    all_hypotheses = []
    
    smoothing_function = SmoothingFunction().method1  # Choose an appropriate smoothing method
    
    with torch.no_grad():
        for batch, (inputs, mask, lab) in enumerate(dataloader):
            inputs = inputs.to(device)
            mask = mask.to(device)
            labels = lab.to(device)
            
            outputs = model(input_ids=inputs, attention_mask=mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            
            preds = torch.argmax(outputs.logits, dim=-1)
            all_preds.extend(preds.cpu().numpy().flatten())
            all_labels.extend(labels.cpu().numpy().flatten())
            # Assuming labels and preds are sequences for BLEU score
            for ref, hyp in zip(labels, preds):
                ref_tokens = ref.cpu().numpy().tolist()
                hyp_tokens = hyp.cpu().numpy().tolist()
                all_references.append([tokenizer.decode(x, skip_special_tokens=True).split()for  x in ref_tokens])
                all_hypotheses.append(tokenizer.decode(hyp_tokens, skip_special_tokens=True).split())
    
    avg_loss = total_loss / len(dataloader)
    
    # Calculate precision, recall, accuracy
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    accuracy = accuracy_score(all_labels, all_preds)
    
    # Calculate BLEU score with smoothing
    bleu_score = corpus_bleu(all_references, all_hypotheses, smoothing_function=smoothing_function)
    
    return avg_loss,precision, recall, accuracy, bleu_score

# Example usage
# Assuming you have the following variables defined:
# model, criterion, val_dataloader, device, tokenizer

avg_loss, precision, recall, accuracy, bleu_score = validate(model, valDataloader, criterion, device, tokenizer)
print(f"Validation loss: {avg_loss:.4f}")
print(f"Validation Precision: {precision:.4f}")
print(f"Validation Recall: {recall:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation BLEU score: {bleu_score:.4f}")


c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Validation loss: 4.2125
Validation Precision: 0.0001
Validation Recall: 0.0002
Validation Accuracy: 0.5287
Validation BLEU score: 0.0000


In [156]:
type(trainData)

torch.utils.data.dataloader.DataLoader

In [157]:
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BlenderbotForCausalLM(
      (model): BlenderbotDecoderWrapper(
        (decoder): BlenderbotDecoder(
          (embed_tokens): BlenderbotScaledWordEmbedding(8008, 1280, padding_idx=0)
          (embed_positions): BlenderbotLearnedPositionalEmbedding(128, 1280)
          (layers): ModuleList(
            (0): BlenderbotDecoderLayer(
              (self_attn): BlenderbotAttention(
                (k_proj): Linear8bitLt(in_features=1280, out_features=1280, bias=True)
                (v_proj): lora.Linear(
                  (base_layer): Linear8bitLt(in_features=1280, out_features=1280, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1280, out_features=64, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (de

In [158]:
for epoch in tqdm.trange(epochs,desc="Epoch(s)"):
    valloss,valacc, valprecision, valrecall, valbleu_score=validate(model, valDataloader, criterion, device, tokenizer)
    train_loss,acc=train(model,optimizer,criterion,trainData)
   
    dataDict={
        "Epoch": epoch+1,
        "Train Loss": train_loss,
        "Train Accuracy":acc,
        "Validation Loss": valloss,
        "Validation Accuracy":valacc,
        "Precision": valprecision, 
        "Recall":valrecall, 
        "Bleu Score":valbleu_score,
    }
    print(dataDict)
    wandb.log(dataDict)
    model.save_pretrained("tunedModels/trainedSmallBlenderBotLoRA - Epoch {}".format(epoch))


Epoch(s):   0%|          | 0/400 [00:00<?, ?it/s]c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\bitsandbytes\autograd\_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier,

{'Epoch': 1, 'Train Loss': 4.183987753496644, 'Train Accuracy': 0.5313754857512953, 'Validation Loss': 4.222996884141087, 'Validation Accuracy': 0.00011642140324440986, 'Precision': 0.0002089427496865859, 'Recall': 0.5287469267598344, 'Bleu Score': 3.459808787090618e-08}


Epoch(s):   0%|          | 1/400 [01:55<12:48:51, 115.62s/it]c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\bitsandbytes\autograd\_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag

{'Epoch': 2, 'Train Loss': 4.1294359894088615, 'Train Accuracy': 0.5313754857512953, 'Validation Loss': 4.156163770305223, 'Validation Accuracy': 0.00012447776696918, 'Precision': 0.00023501762632197415, 'Recall': 0.5287469267598344, 'Bleu Score': 7.017436003425787e-84}


Epoch(s):   0%|          | 2/400 [04:01<13:27:15, 121.70s/it]c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\bitsandbytes\autograd\_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\bitsandbytes\autograd\_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float1

...

Epoch(s): 100%|█████████▉| 398/400 [14:25:26<04:23, 131.88s/it]c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\bitsandbytes\autograd\_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(aver

{'Epoch': 399, 'Train Loss': 1.8168238891257766, 'Train Accuracy': 0.8679309693436961, 'Validation Loss': 1.6610725677210438, 'Validation Accuracy': 0.19582676367315996, 'Precision': 0.13913443258435435, 'Recall': 0.8800304089026915, 'Bleu Score': 0.370999445788102}


Epoch(s): 100%|█████████▉| 399/400 [14:27:38<02:12, 132.07s/it]c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\bitsandbytes\autograd\_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(aver

{'Epoch': 400, 'Train Loss': 1.815944814550284, 'Train Accuracy': 0.8679039831606218, 'Validation Loss': 1.6605953601273624, 'Validation Accuracy': 0.19521091671670854, 'Precision': 0.13934452232252484, 'Recall': 0.8801962021221532, 'Bleu Score': 0.371600905317761}


Epoch(s): 100%|██████████| 400/400 [14:29:50<00:00, 130.48s/it]


In [ ]:
def zip_directory(directory_path, zip_name):
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for folder_name, subfolders, filenames in os.walk(directory_path):
            for filename in filenames:
                file_path = os.path.join(folder_name, filename)
                zipf.write(file_path, os.path.relpath(file_path, directory_path))


In [ ]:
save_directory = "tunedModels/trainedSmallBlenderBotLoRA_Epoch_{}".format(epoch)
zip_name = f'/content/drive/MyDrive/Bbot/{save_directory}.zip'

In [ ]:
zip_directory(save_directory, zip_name)